<a href="https://colab.research.google.com/github/theprashasst/Medical-Reasoning-DeepSeek-R1/blob/main/prototyping_Intern_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:


%%capture

!pip install unsloth # install unsloth
!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git # Also get the latest version Unsloth!

In [2]:
# from google.colab import userdata
# hugging_face_token=userdata.get('HF_TOKEN')
# ngrok_token=userdata.get('NGROK')
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
hugging_face_token=user_secrets.get_secret("HF_TOKEN")
ngrok_token=user_secrets.get_secret("NGROK")



In [3]:
%%capture
!pip install langchain_community transformers fastapi uvicorn pyngrok gradio


In [4]:

from fastapi import FastAPI
from pydantic import BaseModel
# from transformers import pipeline
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from pyngrok import ngrok
import uvicorn
import nest_asyncio
import gradio as gr

In [5]:
from transformers import TextIteratorStreamer
# TextStreamer

In [6]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
# if device=="cpu": 
#     from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
#     print("Unsloth wont work")
# else:
#     # %%capture

#     !pip install unsloth # install unsloth
#     !pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git
#     from unsloth import FastLanguageModel

In [7]:
from unsloth import FastLanguageModel

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [8]:
max_seq_length = 2048 # Define the maximum sequence length a model can handle (i.e. how many tokens can be processed at once)
dtype = None # Set to default
load_in_4bit = True # Enables 4 bit quantization — a memory saving optimization

In [9]:

from peft import PeftModel
import os

def get_model(device,max_seq_length,dtype,load_in_4bit):
    # Define the base model and adapter
    adapter_name = "Prashasst/Medical-Reasoning-DeepSeek-8B" 
    
    temp_dir = "/tmp"
    os.makedirs(temp_dir, exist_ok=True)
    
    
    
    if device=="cpu":
        base_model_name= "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
        # Load base model and tokenizer
        tokenizer = AutoTokenizer.from_pretrained(base_model_name,cache_dir=temp_dir)
        base_model = AutoModelForCausalLM.from_pretrained(base_model_name,cache_dir=temp_dir,
            device_map="auto",  # Load on CPU properly
            offload_folder=temp_dir)
        
        # Load PEFT model
        model = PeftModel.from_pretrained(base_model, adapter_name)
    else:
        # Set parameters
        base_model_name = "unsloth/DeepSeek-R1-Distill-Llama-8B-unsloth-bnb-4bit"
    
    
        # Load the DeepSeek R1 model and tokenizer using unsloth — imported using: from unsloth import FastLanguageModel
        base_model, tokenizer = FastLanguageModel.from_pretrained(
            model_name=base_model_name,  # Load the pre-trained DeepSeek R1 model (8B parameter version)
            max_seq_length=max_seq_length, # Ensure the model can process up to 2048 tokens at once
            dtype=dtype, # Use the default data type (e.g., FP16 or BF16 depending on hardware support)
            load_in_4bit=load_in_4bit, # Load the model in 4-bit quantization to save memory
            token=hugging_face_token, # Use hugging face token
        )
        model = PeftModel.from_pretrained(base_model, adapter_name)
    return model,tokenizer




In [10]:
model,tokenizer= get_model(device,max_seq_length,dtype,load_in_4bit)



==((====))==  Unsloth 2025.2.4: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/52.9k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/824 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/336M [00:00<?, ?B/s]

In [11]:
prompt_style = """
You are an advanced AI Doctor developed by Prashasst (ML engineer). Your expertise spans across medical reasoning, diagnostics, treatments, and healthcare advice.
You provide accurate, well-reasoned, and evidence-based responses while maintaining an empathetic and professional tone.

Before answering, break down the question logically using a step-by-step reasoning approach. Your response should be:
- Factually accurate (based on medical science and clinical guidelines).
- Clear and structured (easily understandable for both medical professionals and general users).
- Ethical and responsible (avoid giving direct medical prescriptions without professional consultation).

---
### 🏥 **Previous Conversation**:
{history}

---
### 👨‍⚕️ **Question**:
{question}

<think>
**Step 1: Understanding the Query**
<Analyze the medical context and key symptoms/disease discussed in the question.>

**Step 2: Possible Causes & Differential Diagnosis**
<Provide a list of potential medical conditions related to the query, explaining the reasoning behind each possibility.>

**Step 3: Medical Explanation**
<Dive deeper into the most likely diagnosis, explaining the biological mechanisms, symptoms, and relevant medical research.>

**Step 4: Suggested Next Steps**
<Recommend general medical advice, possible lab tests, or clinical approaches without acting as a direct replacement for a doctor’s consultation.>

**Step 5: Disclaimer**
*"This information is for educational purposes only and should not replace professional medical advice. Always consult a certified healthcare provider for diagnosis and treatment."*
</think>

### **💡 AI Doctor’s Response**:
{response}
"""


In [ ]:
# type(tokenizer)

In [ ]:
# # Creating a test question for inference
# question = """"help me generate a medical report for doctors appointment?"""

# # Enable optimized inference mode for Unsloth models (improves speed and efficiency)
# FastLanguageModel.for_inference(model)  # Unsloth has 2x faster inference!

# # Format the question using the structured prompt (`prompt_style`) and tokenize it
# inputs = tokenizer([prompt_style.format(history="",question=question,response="")], return_tensors="pt").to(device)  # Convert input to PyTorch tensor & move to GPU
# text_streamer = TextStreamer(tokenizer, skip_prompt=True)
# # Generate a response using the model
# outputs = model.generate(
#     streamer=text_streamer,
#     input_ids=inputs.input_ids, # Tokenized input question
#     attention_mask=inputs.attention_mask, # Attention mask to handle padding
#     max_new_tokens=1200, # Limit response length to 1200 tokens (to prevent excessive output)
#     # use_cache=True, # Enable caching for faster inference
# )

# # Decode the generated output tokens into human-readable text
# # response = tokenizer.batch_decode(outputs)

# # # Extract and print only the relevant response part (after "### Response:")
# # print(response[0].split("### Response:")[1])
# # print("NOw second model with lora \n")
# # FastLanguageModel.for_inference(model2)  # Unsloth has 2x faster inference!

# # # Format the question using the structured prompt (`prompt_style`) and tokenize it
# # inputs2 = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to(device)  # Convert input to PyTorch tensor & move to GPU

# # # Generate a response using the model
# # outputs2 = model2.generate(
# #     input_ids=inputs.input_ids, # Tokenized input question
# #     attention_mask=inputs.attention_mask, # Attention mask to handle padding
# #     max_new_tokens=1200, # Limit response length to 1200 tokens (to prevent excessive output)
# #     use_cache=True, # Enable caching for faster inference
# # )

# # # Decode the generated output tokens into human-readable text
# # response2 = tokenizer.batch_decode(outputs2)

# # # Extract and print only the relevant response part (after "### Response:")
# # print(response2[0].split("### Response:")[1])

In [12]:
from langchain.callbacks.manager import BaseRunManager
from typing import List

class StreamingRunManager(BaseRunManager):
    def __init__(self, run_id, handlers=None, inheritable_handlers=None):
        super().__init__(run_id=run_id, handlers=handlers or [], inheritable_handlers=inheritable_handlers or [])
        self.tokens: List[str] = []  # Store streamed tokens

    def on_llm_new_token(self, token: str) -> None:
        """Capture streamed tokens and store them."""
        self.tokens.append(token)


In [13]:
from langchain.llms.base import LLM
from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.schema import LLMResult
from typing import Optional, List, Any
from unsloth import FastLanguageModel
from transformers import TextStreamer
import torch
from pydantic import Field
from threading import Thread

class UnslothLLM(LLM):
    """Custom LangChain LLM using Unsloth for optimized inference."""

    model: Any = Field(..., description="Pre-loaded Unsloth FastLanguageModel or CasualLLM")
    tokenizer: Any = Field(..., description="Pre-loaded tokenizer")

    def __init__(self, **kwargs):
        """Initialize the model for inference using FastLanguageModel."""
        super().__init__(**kwargs)
        if torch.cuda.is_available():
            FastLanguageModel.for_inference(self.model)  # Enable optimized inference

    def _call(self, prompt: str,stop: Optional[List[str]] = None,
        run_manager: Optional[StreamingRunManager] = None) -> str:
        """Generates text using the Unsloth-optimized model."""
        inputs = self.tokenizer(
            [prompt],
            return_tensors="pt",
            padding=True,
        ).to(device)

        streamer = TextIteratorStreamer(self.tokenizer, skip_prompt=True)
       
        thread = Thread(
        target=self.model.generate,
        kwargs={
            "input_ids": inputs.input_ids,
            "attention_mask": inputs.attention_mask,
            "streamer": streamer,
            "max_new_tokens": 540,
            "pad_token_id": self.tokenizer.eos_token_id,
            })
            # ,daemon=True)
        thread.start()
    
        # Store generated tokens to reconstruct full response
        full_response = ""
        
        # for token in streamer:
        #     full_response += token
        #     yield token  # Stream to Gradio in real time
        for new_text in streamer:
            full_response += new_text
            if run_manager:  # Stream token-by-token to LangChain
                # print(new_text)
                run_manager.on_llm_new_token(new_text)
        # thread.join()
        if full_response.startswith(prompt):
            full_response = full_response[len(prompt):].strip()
        # generated_text= self.tokenizer.decode(full_response, skip_special_tokens=True).strip()
        # return generated_text
        return full_response

    
    
        # Remove any unintended parts (prompt, system instructions)
        # if full_response.startswith(prompt):
        #     full_response = full_response[len(prompt):].strip()
    
        # return full_response

        # text_streamer = TextStreamer(self.tokenizer, skip_prompt=True)

        # outputs = self.model.generate(
        #     input_ids=inputs.input_ids,
        #     attention_mask=inputs.attention_mask,
        #     streamer=text_streamer,
        #     max_new_tokens=1200,
        #     pad_token_id=self.tokenizer.eos_token_id,
        # )

        # generated_text= self.tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
        # return generated_text[len(prompt):].strip()

    @property
    def _llm_type(self) -> str:
        """Return type of llm."""
        return "Prashasst's Custom Class Unsloth"

In [14]:
from langchain.prompts import PromptTemplate

# Match your model's training format (modify accordingly)
prompt_template ="""
You are an advanced AI Doctor developed by Prashasst (ML engineer). Your expertise spans across medical reasoning, diagnostics, treatments, and healthcare advice.
You provide accurate, well-reasoned, and evidence-based responses while maintaining an empathetic and professional tone.

Before answering, break down the question logically using a step-by-step reasoning approach. Your response should be:
- Factually accurate (based on medical science and clinical guidelines).
- Clear and structured (easily understandable for both medical professionals and general users).
- Ethical and responsible (avoid giving direct medical prescriptions without professional consultation).

---
### 🏥 **Previous Conversation**:
{history}

---
### 👨‍⚕️ **Human**:
{input}

<think>
**Step 1: Understanding the Query**
<Analyze the medical context and key symptoms/disease discussed in the question.>

**Step 2: Possible Causes & Differential Diagnosis**
<Provide a list of potential medical conditions related to the query, explaining the reasoning behind each possibility.>

**Step 3: Medical Explanation**
<Dive deeper into the most likely diagnosis, explaining the biological mechanisms, symptoms, and relevant medical research.>

**Step 4: Suggested Next Steps**
<Recommend general medical advice, possible lab tests, or clinical approaches without acting as a direct replacement for a doctor’s consultation.>

**Step 5: Disclaimer**
*"This information is for educational purposes only and should not replace professional medical advice. Always consult a certified healthcare provider for diagnosis and treatment."*
</think>

### **💡 AI Doctor’s Response**:

"""


prompt = PromptTemplate(
    input_variables=["history", "input"],
    template=prompt_template,
)

In [15]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

conversation = ConversationChain(
    llm=UnslothLLM(model=model, tokenizer=tokenizer),
    prompt=prompt,
    memory=memory,
    # verbose=True,  # For debugging
)

<ipython-input-15-51a5e1b7ef0b>:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()
<ipython-input-15-51a5e1b7ef0b>:6: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation = ConversationChain(


In [ ]:
# response = conversation.predict(input="who is prashasst?")
# print(response)
# print(response)

In [16]:
import time
import uuid  # To generate unique IDs
import threading  # Import threading

# Define request format
class ChatInput(BaseModel):
    message: str

def AI_doc(message: ChatInput, history):
    run_manager = StreamingRunManager(run_id=str(uuid.uuid4()))  # Generate a unique run ID

    # Run the LLM in a separate thread
    response_thread = threading.Thread(
        target=conversation.llm._call,
        args=(message,),  # Correctly pass message content
        kwargs={"run_manager": run_manager} # Pass run_manager
        # daemon=True
    )
    response_thread.start()

    # Yield tokens in real time
    prev_len = 0
    outt = ""

    while response_thread.is_alive() or prev_len < len(run_manager.tokens):
        if len(run_manager.tokens) > prev_len:
            outt += run_manager.tokens[prev_len]
            prev_len += 1
            yield outt
        else:
            time.sleep(0.05)  # Prevent CPU overuse

    response_thread.join()  # Ensure the thread has finished execution


In [ ]:
# conversation = ConversationChain(
#     llm=UnslothLLM(model=model, tokenizer=tokenizer),
#     prompt=prompt,
#     memory=memory,
#     # verbose=True,  # For debugging
# )
# def AI_doc( message:ChatInput , history):
#     response = ""  # Store full response
    
#     # Run the conversation chain to get the streamer
#     for new_text in conversation.predict(input=message):  
#         response += new_text  
#         yield response
#     # response=conversation.predict(input=message)
#     # return response
#     # for token in llm._call(prompt, run_manager=None):  # ⚡ Calls the model
#     #     yield token  # 🔥 Streams each token to Gradio in real-time


    
    

In [17]:
app=gr.ChatInterface(AI_doc )
app.launch()

/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py:290: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860
Kaggle notebooks require sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://f8ba766427d0ccfc48.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Exception in thread Thread-25 (_fast_generate):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/_contextlib.py", line 116, in decorate_context
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/unsloth/models/llama.py", line 1562, in _fast_generate
    output = generate(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/peft/peft_model.py", line 1838, in generate
    outputs = self.base_model.generate(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/_contextlib.py", line 116, in decorate_context
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py", line 2255, in generate
    result = self._sample(
  File "/usr/local/

In [ ]:
# Initialize FastAPI app
# app = FastAPI()







In [ ]:
# # API endpoint for chatbot
# @app.post("/chat")
# def chat(res: ChatInput):
#     response = conversation.predict(input=res.message)
#     return {"response": response}

# # Set up ngrok tunnel
# ngrok.set_auth_token(ngrok_token)
# ngrok_tunnel = ngrok.connect(8000)
# print("Public URL:", ngrok_tunnel.public_url)

# # Fix event loop issues in Colab
# nest_asyncio.apply()

# # Run FastAPI server
# uvicorn.run(app, port=8000)
